## Imports

In [34]:
# Operational Packages
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import os


# Directories
nb_dir = Path.cwd()
REPO_ROOT = nb_dir.parent
data_dir = REPO_ROOT / 'data/'
clean_dir = data_dir / 'clean/'
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

## Load Data

In [3]:
data_path = REPO_ROOT/'data/raw/NBAC_summarystats_1972to2024_20250506.xlsx'
NBCA = pd.read_excel(data_path, sheet_name=None)



In [22]:
print(NBCA.keys())

NBCA_admin = NBCA['sumstats_admin']
NBCA_meta = NBCA['metadata']
NBCA_admin2 = NBCA['sumstats_admin2']
NBCA_parks = NBCA['sumstats_natpark']
NBCA_years = NBCA['NBAC_1972_2024_20250506']


dict_keys(['sumstats_admin', 'sumstats_natpark', 'sumstats_admin2', 'NBAC_1972_2024_20250506', 'metadata'])


## Clean Data

### Metadata

In [5]:
NBCA_meta = NBCA_meta['National Burned Area Composite - Metadata']

NBCA_meta

0      Source: https://cwfis.cfs.nrcan.gc.ca/datamart...
1      View complete metadata - https://cwfis.cfs.nrc...
2                                                    NaN
3                                                Summary
4      The NBAC is a national, geospatial burned area...
                             ...                        
155    GH-Gwaii Haanas National Park Reserve and Haid...
156    PRESCRIBED identifies a prescribed fire as rep...
157       VERSION identifies the annual dataset version.
158    GID is a Global Identifier that concatenates t...
159    useful for selecting unique fire records merge...
Name: National Burned Area Composite - Metadata, Length: 160, dtype: object

#### Meta Summary

In [6]:
meta_summary_list = []
for row in NBCA_meta[4:11]:
    meta_summary_list.append(row)
meta_summary = "".join(meta_summary_list)

meta_summary

"The NBAC is a national, geospatial burned area product compiled annually since 1972 forannual estimates of carbon emissions. A rule-based decision process is used to select the bestburned area perimeters from a number of available data providers. // La CNSB est un outil decartographie nationale et géospatiale des superficies brûlées compilé chaque année depuis1972 afin d'en calculer annuellement les émissions de carbone. Un processus décisionnel axésur les règles permet de sélectionner les meilleures fourniseurs de données disponibles pourreprésenter une situation de feu donnée."

#### Meta Description

In [7]:
meta_description = " ".join(NBCA_meta.iloc[np.r_[13:67]].dropna().astype(str))
meta_description

"The National Burned Area Composite (NBAC) is a product created as a component of the Fire Monitoring, Accounting and Reporting System (FireMARS), jointly developed by the Canada Centre for Mapping and Earth Observation (formerly the Canada Centre for Remote Sensing) of Natural Resources Canadaand the Canadian Forest Service. FireMARS was initially developed with funding support from the Canadian Space Agency Government Related Initiatives Program (http://www4.asc-csa.gc.ca/auot-eoau/eng/grip/about.aspx) through a collaboration of those in fire research (http://www.nrcan.gc.ca/forests/fire/13143), forest carbon accounting (http://www.nrcan.gc.ca/forests/climate-change/13087) and remote sensing. NBAC is a national product compiled annually since 1972 by the FireMARS system which tracks forest fires for annual estimates of carbon emissions and to help identify National Forest Inventory plots that may have been disturbed by fire.See the FireMARS website at http://www.nrcan.gc.ca/forests/f

#### Meta Fields & Attributes

In [8]:
# 1. Take just the “fields” section of the metadata
meta_lines = (
    NBCA_meta.iloc[67:]      # start at row 67
    .dropna()                # drop blank lines
    .astype(str).str.strip() # make sure they're strings, strip spaces
)

# (optional) drop the header line if it exists
meta_lines = meta_lines[meta_lines != "Fields / Attributes"]

# 2. Split each line into first word + rest
split = meta_lines.str.split(n=1, expand=True)

# 3. Name the two resulting columns
split.columns = ["Field", "Attribute"]

meta_fields = split.reset_index(drop=True)


In [9]:
meta_fields

,Field,Attribute
0,YEAR,is the fire year
1,NFIREID,is a uniquely assigned ID to each fire event o...
2,specific,year. It is the common ID used to link a fire ...
3,"territorial,",or national park boundary for a specific year.
4,BASRC,identifies the burned area product from a data...
...,...,...
87,GH-Gwaii,Haanas National Park Reserve and Haida Heritag...
88,PRESCRIBED,identifies a prescribed fire as reported by th...
89,VERSION,identifies the annual dataset version.
90,GID,is a Global Identifier that concatenates the f...


In [10]:
def merge_meta_rows(df, main_idx, extra_idxs):
    parts = [str(df.loc[main_idx, "Attribute"]).strip()]
    for i in extra_idxs:
        field_i = str(df.loc[i, "Field"]).strip()
        attr_i  = str(df.loc[i, "Attribute"]).strip()
        parts.append(f"{field_i} {attr_i}")
    df.loc[main_idx, "Attribute"] = " ".join(parts)
    return df.drop(extra_idxs).reset_index(drop=True)

# Example: NFIREID + its two continuation lines
meta_fields = merge_meta_rows(meta_fields, main_idx=1, extra_idxs=[2, 3])
meta_fields = merge_meta_rows(meta_fields, main_idx=6, extra_idxs=[7])
meta_fields = merge_meta_rows(meta_fields, main_idx=8, extra_idxs=[9])
meta_fields = merge_meta_rows(meta_fields, main_idx=10, extra_idxs=[11,12,13])
meta_fields = merge_meta_rows(meta_fields, main_idx=11, extra_idxs=[12,13])
meta_fields = merge_meta_rows(meta_fields, main_idx=12, extra_idxs=[13])
meta_fields = merge_meta_rows(meta_fields, main_idx=15, extra_idxs=[16])
meta_fields = merge_meta_rows(meta_fields, main_idx=79, extra_idxs=[80])


In [ ]:
def split_field_dash(df, field_col="Field", attr_col="Attribute"):
    # 1. rows where the Field contains a dash
    mask = df[field_col].astype(str).str.contains("-", na=False)

    # 2. split on the first '-' into two parts
    split = df.loc[mask, field_col].astype(str).str.split("-", n=1, expand=True)
    split.columns = ["abbr", "name_part"]

    # 3. clean up pieces
    abbr = split["abbr"].str.strip()
    name_part = split["name_part"].fillna("").str.strip()   # text after '-'
    existing_attr = df.loc[mask, attr_col].fillna("").str.strip()

    # 4. combine name_part + existing_attr (add a space only if both non-empty)
    glue = np.where((name_part != "") & (existing_attr != ""), " ", "")
    combined_attr = (name_part + glue + existing_attr).str.strip()

    # 5. write back into the original dataframe
    df.loc[mask, field_col] = abbr
    df.loc[mask, attr_col] = combined_attr

    return df

meta_fields = split_field_dash(meta_fields)


In [25]:
meta_fields.iloc[25:30]

,Field,Attribute
25,ON,Ontario
26,PC,Parks Canada
27,PE,Prince Edward Island
28,QC,"Quebec,"
29,SK,Saskatchewan


#### Provincial/Territory Summary Statistics

In [20]:
# 1. Save the description from the first column name
admin_description = NBCA_admin.columns[0]

# 2. Drop the first row (all NaN / banner)
NBCA_summary_stats = NBCA_admin.iloc[1:].copy()

# 3. Use the next row as header
NBCA_summary_stats.columns = NBCA_summary_stats.iloc[0]

# 4. Drop that header row from the data and reset index
NBCA_summary_stats = NBCA_summary_stats.iloc[1:].reset_index(drop=True)

NBCA_summary_stats.attrs['description'] = admin_description

NBCA_summary_stats.head(5)

1,YEAR,AB,BC,MB,NB,NL,NS,NT,NU,ON,PC,PE,QC,SK,YT,CANADA
0,2024,719604.355673,911524.79932,233958.983969,7938.424448,60385.50932,46.177906,1647294.00149,42.072809,79874.038884,92589.440537,21.051528,208466.217566,760954.259178,181700.081212,4904399.41384
1,2023,1950086.59448,2209978.837971,142526.442937,2277.911368,18934.132477,21809.410837,3487336.802179,3784.6894,338450.972125,774388.558646,NaN,4266657.231382,1086507.599438,333095.330548,14635834.513788
2,2022,119898.638877,114453.97964,135861.741668,NaN,20082.396889,2845.138552,579870.789403,987.072408,3312.651189,34323.808935,NaN,28199.857015,224706.437495,194283.416626,1458825.928697
3,2021,56143.805131,785897.164377,1139281.668036,NaN,NaN,184.381283,154455.192913,247.075691,687831.085672,43060.129092,NaN,49363.933784,867993.573556,133945.834348,3918403.843883
4,2020,2830.611904,13761.809879,43146.398874,NaN,2720.240278,672.3788,18156.350975,49.16431,13185.115149,2280.475027,NaN,51483.340908,42739.985624,15939.710809,206965.582537


In [19]:
dataset_meta = {
    "name": "NBAC burned area by administrative area",
    "description": admin_description,
    "source": "https://cwfis.cfs.nrcan.gc.ca",  # example
    "units": "adjusted hectares",
}


In [35]:

# 1. Save the description (first non-NaN banner row)
parks_description = NBCA_parks.iloc[1, 0]   # "Sum of SUM_ADJ_HA"

# 2. Drop top two rows; row 2 becomes header
NBAC_parks_clean = NBCA_parks.iloc[2:].copy()

# set header from the first remaining row
NBAC_parks_clean.columns = NBAC_parks_clean.iloc[0]

# 3. Drop that header row from the data and reset index
NBAC_parks_clean = NBAC_parks_clean.iloc[1:-1].reset_index(drop=True)

# 4. Rename the first column from "Row Labels" to YEAR (they’re years)
NBAC_parks_clean = NBAC_parks_clean.rename(columns={"Row Labels": "YEAR"})

# (optional) convert YEAR to integer
NBAC_parks_clean["YEAR"] = NBAC_parks_clean["YEAR"].astype(int)

# 5. Attach the description as metadata (like alt text)
NBAC_parks_clean.attrs["description"] = parks_description

In [36]:
NBAC_parks_clean

2,YEAR,AB,BC,MB,NB,NL,NS,NT,NU,ON,PC,PE,QC,SK,YT,Grand Total
0,2024,719604.355673,911524.79932,233958.983969,7938.424448,60385.50932,46.177906,1647294.00149,42.072809,79874.038884,92589.440537,21.051528,208466.217566,760954.259178,181700.081212,4904399.41384
1,2023,1950086.59448,2209978.837971,142526.442937,2277.911368,18934.132477,21809.410837,3487336.802179,3784.6894,338450.972125,774388.558646,NaN,4266657.231382,1086507.599438,333095.330548,14635834.513788
2,2022,119898.638877,114453.97964,135861.741668,NaN,20082.396889,2845.138552,579870.789403,987.072408,3312.651189,34323.808935,NaN,28199.857015,224706.437495,194283.416626,1458825.928697
3,2021,56143.805131,785897.164377,1139281.668036,NaN,NaN,184.381283,154455.192913,247.075691,687831.085672,43060.129092,NaN,49363.933784,867993.573556,133945.834348,3918403.843883
4,2020,2830.611904,13761.809879,43146.398874,NaN,2720.240278,672.3788,18156.350975,49.16431,13185.115149,2280.475027,NaN,51483.340908,42739.985624,15939.710809,206965.582537
5,2019,711139.028585,20353.85822,71850.15939,NaN,379.29833,NaN,110103.87742,2296.489088,225155.388524,125191.826876,NaN,8922.125169,55443.066691,272855.490016,1603690.608309
6,2018,47794.571934,1045479.466827,210894.190262,70.485825,15.129373,257.677607,14167.361917,278.98713,221010.673198,46650.732838,NaN,83787.242188,108944.977945,79858.90342,1859210.400464
7,2017,45255.410385,1029458.854195,217522.342579,354.04303,NaN,784.735991,814013.72961,139.64454,94933.345368,117243.809665,NaN,30625.561844,370530.536836,328084.631454,3048946.645497
8,2016,504329.335133,79045.472882,34591.372498,NaN,2870.825652,794.521453,213972.810825,142.253915,77794.335206,18372.919966,NaN,35589.955679,227122.66331,19066.081021,1213692.54754
9,2015,413477.375791,231409.922853,65531.124651,295.638263,3432.555231,562.895106,526643.081097,73.369216,38314.090128,457058.495323,NaN,5394.511517,1461961.507993,147642.977597,3351797.544766


In [38]:
# starting point: NBCA_years = NBCA['NBAC_1972_2024_20250506'] (or whatever name)

# 1. Capture the description from the first few rows of the first column
years_description = (
    NBCA_years.iloc[0:3, 0]      # rows 0–2, first column
    .dropna()
    .astype(str)
    .str.strip()
    .str.join(" ")               # join into one sentence; use "\n".join(...) for line breaks
)

# 2. Drop the first 3 rows; row index 3 becomes the header row
years_clean = NBCA_years.iloc[3:].copy()

# set header from the first remaining row
years_clean.columns = years_clean.iloc[0]

# 3. Drop that header row from the data and reset index
years_clean = years_clean.iloc[1:].reset_index(drop=True)

# optional: attach description as metadata (like "alt text")
years_clean.attrs["description"] = years_description

# optional: make YEAR numeric
years_clean["YEAR"] = years_clean["YEAR"].astype(int)


In [41]:
years_clean

3,YEAR,NFIREID,BASRC,FIREMAPS,FIREMAPM,FIRECAUS,HS_SDATE,HS_EDATE,AG_SDATE,AG_EDATE,CAPDATE,POLY_HA,ADJ_HA,ADJ_FLAG,ADMIN_AREA,NATPARK,PRESCRIBED,VERSION,GID
0,2024,1,MAFiMS,Landsat,Processed imagery,Natural,2024-08-12 00:00:00,2024-09-02 00:00:00,2024-07-29 00:00:00,2024-10-04 00:00:00,2024-09-15 00:00:00,625.258308,625.258308,NaN,NT,NaN,NaN,20250506,2024_1
1,2024,2,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-06-29 00:00:00,2024-08-12 00:00:00,2024-06-28 00:00:00,2024-10-04 00:00:00,2024-08-22 00:00:00,185.715925,185.715925,NaN,NT,NaN,NaN,20250506,2024_2
2,2024,3,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-07-17 00:00:00,2024-07-24 00:00:00,2024-07-17 00:00:00,2024-10-04 00:00:00,2024-08-08 00:00:00,1188.87817,1188.87817,NaN,NT,NaN,NaN,20250506,2024_3
3,2024,4,MAFiMS,Landsat,Processed imagery,Natural,2024-07-17 00:00:00,2024-08-13 00:00:00,2024-07-17 00:00:00,2024-10-04 00:00:00,2024-08-30 00:00:00,3862.626905,3862.626905,NaN,NT,NaN,NaN,20250506,2024_4
4,2024,5,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-07-28 00:00:00,2024-08-12 00:00:00,2024-07-31 00:00:00,2024-10-04 00:00:00,2024-08-17 00:00:00,26.552612,26.552612,NaN,NT,NaN,NaN,20250506,2024_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50826,1972,330,MAFiMS,Landsat MSS,Processed imagery,Undetermined,NaN,NaN,NaN,NaN,1972-08-28 00:00:00,2803.592459,2803.592459,NaN,MB,NaN,NaN,20250506,1972_330
50827,1972,331,MAFiMS,Landsat MSS,Processed imagery,Undetermined,NaN,NaN,NaN,NaN,1973-08-23 00:00:00,267.826703,267.826703,NaN,QC,NaN,NaN,20250506,1972_331
50828,1972,332,MAFiMS,Landsat MSS,Processed imagery,Undetermined,NaN,NaN,NaN,NaN,1973-08-23 00:00:00,1395.369141,1395.369141,NaN,QC,NaN,NaN,20250506,1972_332
50829,1972,333,MAFiMS,Landsat MSS,Processed imagery,Undetermined,NaN,NaN,NaN,NaN,1972-07-31 00:00:00,671.96671,671.96671,NaN,QC,NaN,NaN,20250506,1972_333
